In [1]:
!pip install -U transformers datasets evaluate accelerate
!pip install scikit-learn
!pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
 

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

import evaluate
import glob
import numpy as np

In [3]:
BATCH_SIZE = 32
NUM_PROCS = 32
LR = 0.00005
EPOCHS = 5
MODEL = 'bert-base-uncased'
OUT_DIR = 'arxiv_bert'

In [4]:
train_dataset = load_dataset("ccdv/arxiv-classification", split='train[0:3000]')
valid_dataset = load_dataset("ccdv/arxiv-classification", split='validation[0:1000]')
test_dataset = load_dataset("ccdv/arxiv-classification", split='test[0:1000]')
print(train_dataset)
print(valid_dataset)
print(test_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for ccdv/arxiv-classification contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/arxiv-classification.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 3000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})


In [5]:
# Visualize a sample.
train_dataset[0]

{'text': 'Constrained Submodular Maximization via a\nNon-symmetric Technique\n\narXiv:1611.03253v1 [cs.DS] 10 Nov 2016\n\nNiv Buchbinder∗\n\nMoran Feldman†\n\nNovember 11, 2016\n\nAbstract\nThe study of combinatorial optimization problems with a submodular objective has attracted\nmuch attention in recent years. Such problems are important in both theory and practice because\ntheir objective functions are very general. Obtaining further improvements for many submodular\nmaximization problems boils down to finding better algorithms for optimizing a relaxation of\nthem known as the multilinear extension.\nIn this work we present an algorithm for optimizing the multilinear relaxation whose guarantee improves over the guarantee of the best previous algorithm (which was given by Ene\nand Nguyen (2016)). Moreover, our algorithm is based on a new technique which is, arguably,\nsimpler and more natural for the problem at hand. In a nutshell, previous algorithms for this\nproblem rely on symmet

In [6]:
id2label = {
    0: "math.AC",
    1: "cs.CV",
    2: "cs.AI",
    3: "cs.SY",
    4: "math.GR",
    5: "cs.CE",
    6: "cs.PL",
    7: "cs.IT",
    8: "cs.DS",
    9: "cs.NE",
    10: "math.ST"
}
label2id = {
    "math.AC": 0,
    "cs.CV": 1,
    "cs.AI": 2,
    "cs.SY": 3,
    "math.GR": 4,
    "cs.CE": 5,
    "cs.PL": 6,
    "cs.IT": 7,
    "cs.DS": 8,
    "cs.NE": 9,
    "math.ST": 10
}

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
    )

In [10]:
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

tokenized_valid = valid_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=BATCH_SIZE,
    num_proc=NUM_PROCS
)

Map (num_proc=32):   0%|          | 0/3000 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=32):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
tokenized_sample = preprocess_function(train_dataset[0])
print(tokenized_sample)
print(f"Length of tokenized IDs: {len(tokenized_sample.input_ids)}")
print(f"Length of attention mask: {len(tokenized_sample.attention_mask)}")


{'input_ids': [101, 27570, 4942, 5302, 8566, 8017, 20446, 3989, 3081, 1037, 2512, 1011, 19490, 6028, 12098, 9048, 2615, 1024, 28769, 1012, 6021, 17788, 2509, 2615, 2487, 1031, 20116, 1012, 16233, 1033, 2184, 13292, 2355, 9152, 2615, 20934, 2818, 8428, 4063, 30125, 17866, 26908, 1526, 2281, 2340, 1010, 2355, 10061, 1996, 2817, 1997, 22863, 23207, 4818, 20600, 3471, 2007, 1037, 4942, 5302, 8566, 8017, 7863, 2038, 6296, 2172, 3086, 1999, 3522, 2086, 1012, 2107, 3471, 2024, 2590, 1999, 2119, 3399, 1998, 3218, 2138, 2037, 7863, 4972, 2024, 2200, 2236, 1012, 11381, 2582, 8377, 2005, 2116, 4942, 5302, 8566, 8017, 20446, 3989, 3471, 26077, 2015, 2091, 2000, 4531, 2488, 13792, 2005, 23569, 27605, 6774, 1037, 23370, 1997, 2068, 2124, 2004, 1996, 4800, 4179, 2906, 5331, 1012, 1999, 2023, 2147, 2057, 2556, 2019, 9896, 2005, 23569, 27605, 6774, 1996, 4800, 4179, 2906, 23370, 3005, 11302, 24840, 2058, 1996, 11302, 1997, 1996, 2190, 3025, 9896, 1006, 2029, 2001, 2445, 2011, 4372, 2063, 1998, 16577, 1

In [13]:
tokenized_sample = preprocess_function(train_dataset[0])
print(tokenized_sample)

{'input_ids': [101, 27570, 4942, 5302, 8566, 8017, 20446, 3989, 3081, 1037, 2512, 1011, 19490, 6028, 12098, 9048, 2615, 1024, 28769, 1012, 6021, 17788, 2509, 2615, 2487, 1031, 20116, 1012, 16233, 1033, 2184, 13292, 2355, 9152, 2615, 20934, 2818, 8428, 4063, 30125, 17866, 26908, 1526, 2281, 2340, 1010, 2355, 10061, 1996, 2817, 1997, 22863, 23207, 4818, 20600, 3471, 2007, 1037, 4942, 5302, 8566, 8017, 7863, 2038, 6296, 2172, 3086, 1999, 3522, 2086, 1012, 2107, 3471, 2024, 2590, 1999, 2119, 3399, 1998, 3218, 2138, 2037, 7863, 4972, 2024, 2200, 2236, 1012, 11381, 2582, 8377, 2005, 2116, 4942, 5302, 8566, 8017, 20446, 3989, 3471, 26077, 2015, 2091, 2000, 4531, 2488, 13792, 2005, 23569, 27605, 6774, 1037, 23370, 1997, 2068, 2124, 2004, 1996, 4800, 4179, 2906, 5331, 1012, 1999, 2023, 2147, 2057, 2556, 2019, 9896, 2005, 23569, 27605, 6774, 1996, 4800, 4179, 2906, 23370, 3005, 11302, 24840, 2058, 1996, 11302, 1997, 1996, 2190, 3025, 9896, 1006, 2029, 2001, 2445, 2011, 4372, 2063, 1998, 16577, 1

In [14]:
accuracy = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=11,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=3,
    report_to='tensorboard',
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

history = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.887591,0.753000
2,No log,0.771153,0.775000
3,No log,0.797929,0.777000
4,No log,0.752706,0.784000
5,No log,0.771765,0.794000
